In [1]:
import json 
from rake_nltk import Rake
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer 
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
from pyemd import emd
# from gensim.similarities import WmdSimilarity


ModuleNotFoundError: No module named 'fuzzywuzzy'

In [2]:
# drive.mount('/content/drive')

In [3]:
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None

In [4]:
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word)[0]
    except:
        return None

In [5]:
def sentence_similarity(model_key, student_key):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    lemmatizer = WordNetLemmatizer() 

    model_answer_tokenized = word_tokenize(model_key)
    model_pos_tagged = pos_tag(model_answer_tokenized)
    model_tagged = [(lemmatizer.lemmatize(word[0].lower(),penn_to_wn(word[1])),word[1]) for word in model_pos_tagged]
#     model_lemmatized = [lemmatizer.lemmatize(word[0].lower(),penn_to_wn(word[1])) for word in model_pos_tagged]
#     model_tagged = pos_tag(model_lemmatized)
#     print(model_tagged)
#     model_tagged = pos_tag(model_answer_words)

    student_answer_tokenized = word_tokenize(student_key)
    student_pos_tagged = pos_tag(student_answer_tokenized)
    student_tagged = [(lemmatizer.lemmatize(word[0].lower(),penn_to_wn(word[1])),word[1]) for word in student_pos_tagged]
#     student_lemmatized = [lemmatizer.lemmatize(word[0].lower(),penn_to_wn(word[1])) for word in student_pos_tagged]
#     student_tagged = pos_tag(student_lemmatized)
#     print(student_tagged)
    
#     model_tagged = pos_tag(lemmatizer.lemmatize(word_tokenize(model_key)))
#     student_tagged = pos_tag(lemmatizer.lemmatize(word_tokenize(student_key)))
 
    # Get the synsets for the tagged words
    model_syn = [tagged_to_synset(*tagged_word) for tagged_word in model_tagged]
    student_syn = [tagged_to_synset(*tagged_word) for tagged_word in student_tagged]
 
    # Filter out the Nones
    model_syn_filtered = [ss for ss in model_syn if ss]
    student_syn_filtered = [ss for ss in student_syn if ss]
    print(model_syn_filtered, student_syn_filtered)
 
    score, count = 0.0, 0

    for word in model_syn_filtered:
        # Get the similarity value of the most similar word in the other sentence
#         scores = [word.path_similarity(ss) for ss in student_syn_filtered]
        scores = [word.wup_similarity(ss) for ss in student_syn_filtered]
#         print(scores)
        scores = [s for s in scores if s]
        print(scores)
        if len(scores)!=0:
            best_score = max(scores)
            score += best_score
            count += 1
 
    # Average the values
    if count!=0:
        score /= count
        return score
    return 0

In [6]:
answers1 = []
f = open('answers1.txt')
lines = f.readlines()

for line in lines:
    if line!='\n':
        answers1 += [line.replace('\n', '')]
# print(answers1)  

In [7]:
with open('model_answers.txt') as f: 
    data = f.read() 
  
print("Data type before reconstruction : ", type(data)) 
      
# reconstructing the data as a dictionary 
model_answer_list = json.loads(data) 
  
print("Data type after reconstruction : ", type(model_answer_list)) 
# print(model_answer_list) 

Data type before reconstruction :  <class 'str'>
Data type after reconstruction :  <class 'list'>


In [8]:
model_answer = model_answer_list[-2]
student_answer =  answers1[-2]

print(model_answer)
print(student_answer)

{'Enhance the responsiveness to the users.': 0.5, 'Resource sharing within the process.': 0.5, 'Economical': 0.5, 'Completely utilize the multiprocessing architecture.': 0.5}
Responsiveness, Resource Sharing, Economy, Scalability


In [14]:
r = Rake()
lemmatizer = WordNetLemmatizer() 
r.extract_keywords_from_text(student_answer)
student_keywords = r.get_ranked_phrases()
print(student_keywords)
for sentence in model_answer.keys():
    r.extract_keywords_from_text(sentence)
    sentence_keywords = r.get_ranked_phrases()
    print(sentence_keywords)
    for sentence_keyword in sentence_keywords:
        for student_keyword in student_keywords:
            print(lemmatizer.lemmatize([x for x in word_tokenize(sentence_keyword)]))
            print(sentence_keyword,",",student_keyword,",",sentence_similarity(sentence_keyword, student_keyword))

['resource sharing', 'scalability', 'responsiveness', 'economy']
['users', 'responsiveness', 'enhance']


TypeError: unhashable type: 'list'

In [54]:
# r = Rake()
# r.extract_keywords_from_text(student_answer)
# student_keywords = r.get_ranked_phrases()
# grand_total = 0
# for sentence in model_answer.keys():
#     r.extract_keywords_from_text(sentence)
#     sentence_keywords = r.get_ranked_phrases()
#     total = 0
#     for sentence_keyword in sentence_keywords:
#         best =  None
#         max_score = 0
#         for student_keyword in student_keywords:
#             if sentence_similarity(sentence_keyword, student_keyword)>max_score:
#                 max_score = sentence_similarity(sentence_keyword, student_keyword)
#                 best = student_keyword
#         total+=max_score
#         print(sentence_keyword,",",best,",",max_score)
#     print(total/len(sentence_keywords))
#     grand_total += total/len(sentence_keywords)

# print(grand_total/len(model_answer.keys()))

[Synset('new.a.01'), Synset('procedure.n.01')] [Synset('delay.n.01')]
[Synset('new.a.01'), Synset('procedure.n.01')] [Synset('delay.n.01')]
[Synset('new.a.01'), Synset('procedure.n.01')] [Synset('end.v.02')]
[Synset('new.a.01'), Synset('procedure.n.01')] [Synset('run.n.01')]
[Synset('new.a.01'), Synset('procedure.n.01')] [Synset('run.n.01')]
[Synset('new.a.01'), Synset('procedure.n.01')] [Synset('ready.n.01')]
[Synset('new.a.01'), Synset('procedure.n.01')] [Synset('new.a.01')]
[Synset('new.a.01'), Synset('procedure.n.01')] [Synset('new.a.01')]
new process , new , 1.0
1.0
[Synset('run.n.01'), Synset('procedure.n.01')] [Synset('delay.n.01')]
[Synset('run.n.01'), Synset('procedure.n.01')] [Synset('delay.n.01')]
[Synset('run.n.01'), Synset('procedure.n.01')] [Synset('end.v.02')]
[Synset('run.n.01'), Synset('procedure.n.01')] [Synset('run.n.01')]
[Synset('run.n.01'), Synset('procedure.n.01')] [Synset('run.n.01')]
[Synset('run.n.01'), Synset('procedure.n.01')] [Synset('ready.n.01')]
[Synset(